# 汉明重量幅度

支持的设备:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

In [ ]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
CRYPTO_TARGET = 'TINYAES128C'
num_traces = 50

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../../hardware/victims/firmware/simpleserial-aes
make PLATFORM=$1 CRYPTO_TARGET=$2

## 数据追踪理论

在上一个教程中，我们看到一个设备的功率测量是如何与汉明重量相关的。让我们用这个来看看一些任意的数据在哪里被设备处理。稍后我们将在此基础上扩展，考虑一些随机数据和噪音的情况。

我们的目标很简单——我们将在一个地方发送一些全部为1的数据，然后再发送一些全部为0的数据。

## 捕获能力轨迹

捕获功率跟踪将与以前的教程非常相似，只是这次我们将使用一个循环来捕获多个跟踪，以及使用numpy来存储这些跟踪。

### 设置

我们将使用一些辅助脚本来使设置和编程更容易。如果你使用的是XMEGA或STM（CWLITEARM）目标，带有正确的二进制文件应该为你设置。

In [ ]:
%run "../Helper_Scripts/Setup_Generic.ipynb"

In [ ]:
fw_path = '../../hardware/victims/firmware/simpleserial-aes/simpleserial-aes-{}.hex'.format(PLATFORM)

In [ ]:
# program the target
cw.program_target(scope, prog, fw_path)

### 捕获轨迹

下面你可以看到捕获循环。循环的主体加载一些新的明文，设置域端，发送密钥和明文，然后最后记录并将我们的新跟踪附加到`traces[]`列表中。在最后，我们将跟踪数据转换为numpy数组。

In [ ]:
#Capture Traces
from tqdm import tnrange
import numpy as np
import time

ktp = cw.ktp.Basic()

traces = []

for i in tnrange(num_traces, desc='Capturing traces'):
    key, text = ktp.next()
    
    #Currently ALL bits are random. Let's extend bit 0 to a full byte to give us a random 0xFF or 0x00
    if text[0] & 0x01:
        text[0] = 0xFF
    else:
        text[0] = 0x00
    
    trace = cw.capture_trace(scope, target, text, key)
    if trace is None:
        continue
    traces.append(trace)

现在我们有了我们的轨迹，我们也可以用Bokeh绘制它们：

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()
p = figure()

xrange = range(len(traces[0].wave))
p.line(xrange, traces[2].wave, line_color="red")
show(p)

In [ ]:
# cleanup the connection to the target and scope
scope.dis()
target.dis()

## 轨迹分析

### 比较0xFF和0x00

现在我们有了一些轨迹，让我们看看我们实际记录了什么。我们将做以下工作：

1. 将轨迹分到两组：0x00和0xFF
2. 计算每组的平均值。
3. 减去这两个平均数，看看有什么不同。

这将显示在以下两个单元格中。注意0xFF和0x00的数量并不完全是50/50。这就是为什么我们需要确保对它们进行平均。

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
import numpy as np

output_notebook()
p = figure()

one_list = []
zero_list = []

for trace in traces:
    if trace.textin[0] == 0x00:
        one_list.append(trace.wave)
    else:
        zero_list.append(trace.wave)

print("Number of 0xFF: " + str(len(one_list)))
print("Number of 0x00: " + str(len(zero_list)))

In [ ]:
one_avg = np.asarray(one_list).mean(axis=0)
zero_avg = np.asarray(zero_list).mean(axis=0)

diff = one_avg - zero_avg
p.line(range(0, len(traces[0].wave)), diff)
show(p)

显然我们可以看到一个尖峰数据，这说明了HW值的影响幅度。如果你有时间，你也可以尝试以下一些方法：

* 强制输出数据都为0xFF或都为0x00s
* 强制一些中间值都为0xFF或都为0x00s
* 设置多个字节为0xFF或0x00
* 按顺序绘制每个字节，以查看数据移动情况